In [29]:
import io
import os
import re
import sys
import time
import numpy as np

import unicodedata

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

### Import Data + Preprocessing

In [25]:

with open('sas_high.csv', 'r', newline='\n') as f:
    readList = f.read().split('\n')
    highScores = [x.strip() for x in readList]
    
with open('sas_low.csv', 'r', newline='\n') as f:
    readList = f.read().split('\n')
    lowScores = [x.strip() for x in readList]

In [71]:
#DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['sas_high.csv', 'sas_low.csv']

parent_dir = os.path.dirname(".")
parent_dir

''

In [72]:
def labeler(example, index):
    return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
    lines_dataset = tf.data.TextLineDataset(file_name)
    labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
    labeled_data_sets.append(labeled_dataset)
    
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
    all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

for ex in all_labeled_data.take(5):
    print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'1 Let the samples sit to dry for different amount of time.'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'They send the message through the body.'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'2.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Also to say how much weight to the clamp.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'There is a connection between them.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


### Vocab and Embeddings

In [73]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
    some_tokens = tokenizer.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

7400

In [74]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [75]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)
encoded_example = encoder.encode(example_text)
print(encoded_example)

b'1 Let the samples sit to dry for different amount of time.'
[266, 895, 7233, 5954, 2110, 99, 2764, 5262, 5757, 4734, 4858, 5662]


In [76]:
char2idx = {u:i for i, u in enumerate(vocabulary_set)}
idx2char = np.array(vocabulary_set)

# text_as_int = np.array([char2idx[c] for c in text])

In [77]:
def encode(text_tensor, label):
    encoded_text = encoder.encode(text_tensor.numpy())
    return encoded_text, label

In [78]:
def encode_map_fn(text, label):
    # py_func doesn't set the shape of the returned tensors.
    encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

    # `tf.data.Datasets` work best if all components have a shape set
    #  so set the shapes manually: 
    encoded_text.set_shape([None])
    label.set_shape([])

    return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

# You want to use Dataset.map to apply this function to each element of the dataset. Dataset.map runs in graph mode.

#     Graph tensors do not have a value.
#     In graph mode you can only use TensorFlow Ops and functions.

# So you can't .map this function directly: You need to wrap it in a tf.py_function. The tf.py_function will pass regular tensors (with a value and a .numpy() method to access it), to the wrapped python function.


In [79]:
all_encoded_data

<MapDataset shapes: ((None,), ()), types: (tf.int64, tf.int64)>

In [80]:
for element in all_encoded_data: 
    print(element) 
    break


(<tf.Tensor: shape=(12,), dtype=int64, numpy=
array([ 266,  895, 7233, 5954, 2110,   99, 2764, 5262, 5757, 4734, 4858,
       5662])>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)


In [64]:
### Now turning to Keras vectors

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 1000  # Number of samples to train on.
# Path to the data txt file on disk.

lowScores
highScores_k = list(map(lambda x: '\t' + x + '\n', highScores))
highScores_k

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

### Split Data

In [81]:
## have to pay attention to padded shapes - see above for shape\
## padding adds 0's to tokenize sentences so all data the same shape

train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=((None,), ()))


test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=((None,), ()))

In [82]:
## since we added 0
vocab_size += 1


In [83]:
test_data

<PaddedBatchDataset shapes: ((None, None), (None,)), types: (tf.int64, tf.int64)>


### Model Build (Bidirectional LSTM)

In [55]:
model = tf.keras.Sequential()

In [56]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

In [57]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

In [58]:
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3))

In [59]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [60]:
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
697/697 [==============================] - 87s 124ms/step - loss: 0.5316 - accuracy: 0.7387 - val_loss: 0.3886 - val_accuracy: 0.8242
Epoch 2/3
697/697 [==============================] - 81s 116ms/step - loss: 0.3031 - accuracy: 0.8674 - val_loss: 0.3577 - val_accuracy: 0.8364
Epoch 3/3
697/697 [==============================] - 61s 88ms/step - loss: 0.2307 - accuracy: 0.8994 - val_loss: 0.3811 - val_accuracy: 0.8384


### Model Build (Encoder-Decoder GRU RNN)

In [84]:
model = tf.keras.Sequential()

In [85]:
# Add an Embedding layer expecting input vocab size, and
# output embedding dimension of size 64.

model.add(tf.keras.layers.Embedding(vocab_size, 64))

In [86]:
# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model.add(layers.GRU(256, return_sequences=True))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model.add(layers.SimpleRNN(128))

model.add(layers.Dense(10))

model.summary() 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 64)          473664    
_________________________________________________________________
gru_1 (GRU)                  (None, None, 256)         247296    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               49280     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 771,530
Trainable params: 771,530
Non-trainable params: 0
_________________________________________________________________


In [65]:

encoder_vocab = vocab_size
decoder_vocab = vocab_size

encoder_input = layers.Input(shape=(None, ))
encoder_embedded = layers.Embedding(input_dim=encoder_vocab, output_dim=64)(encoder_input)

# Return states in addition to output
output, state_h, state_c = layers.LSTM(
    64, return_state=True, name='encoder')(encoder_embedded)
encoder_state = [state_h, state_c]

decoder_input = layers.Input(shape=(None, ))
decoder_embedded = layers.Embedding(input_dim=decoder_vocab, output_dim=64)(decoder_input)

# Pass the 2 states to a new LSTM layer, as initial state
decoder_output = layers.LSTM(
    64, name='decoder')(decoder_embedded, initial_state=encoder_state)
output = layers.Dense(10)(decoder_output)

model = tf.keras.Model([encoder_input, decoder_input], output)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     473600      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     473600      input_2[0][0]                    
______________________________________________________________________________________________

In [110]:
inputs = np.random.random([32, 10, 8]).astype(np.float32)
gru = tf.keras.layers.GRU(4)

output = gru(inputs)  # The output has shape `[32, 4]`.

gru = tf.keras.layers.GRU(64, return_sequences=True, return_state=True)

# whole_sequence_output has shape `[32, 10, 4]`.
# final_state has shape `[32, 4]`.
whole_sequence_output, final_state = gru(inputs)


In [114]:
gru

### Model Build (Keras Based not from docs)
See http://alexadam.ca/ml/2017/05/05/keras-vae.html

In [ ]:
class VAE(object):
    def create(self, vocab_size=500, max_length=300, latent_rep_size=200):
        self.encoder = None
        self.decoder = None
        self.sentiment_predictor = None
        self.autoencoder = None

        x = Input(shape=(max_length,))
        x_embed = Embedding(vocab_size, 64, input_length=max_length)(x)

        vae_loss, encoded = self._build_encoder(x_embed, latent_rep_size=latent_rep_size, max_length=max_length)
        self.encoder = Model(inputs=x, outputs=encoded)

        encoded_input = Input(shape=(latent_rep_size,))
        predicted_sentiment = self._build_sentiment_predictor(encoded_input)
        self.sentiment_predictor = Model(encoded_input, predicted_sentiment)

        decoded = self._build_decoder(encoded_input, vocab_size, max_length)
        self.decoder = Model(encoded_input, decoded)

        self.autoencoder = Model(inputs=x, outputs=[self._build_decoder(encoded, vocab_size, max_length), self._build_sentiment_predictor(encoded)])
        self.autoencoder.compile(optimizer='Adam',
                                 loss=[vae_loss, 'binary_crossentropy'],
                                 metrics=['accuracy'])

In [ ]:
def _build_encoder(self, x, latent_rep_size=200, max_length=300, epsilon_std=0.01):
    h = Bidirectional(LSTM(500, return_sequences=True, name='lstm_1'), merge_mode='concat')(x)
    h = Bidirectional(LSTM(500, return_sequences=False, name='lstm_2'), merge_mode='concat')(h)
    h = Dense(435, activation='relu', name='dense_1')(h)

    def sampling(args):
        z_mean_, z_log_var_ = args
        batch_size = K.shape(z_mean_)[0]
        epsilon = K.random_normal(shape=(batch_size, latent_rep_size), mean=0., stddev=epsilon_std)
        return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

    z_mean = Dense(latent_rep_size, name='z_mean', activation='linear')(h)
    z_log_var = Dense(latent_rep_size, name='z_log_var', activation='linear')(h)

    def vae_loss(x, x_decoded_mean):
        x = K.flatten(x)
        x_decoded_mean = K.flatten(x_decoded_mean)
        xent_loss = max_length * objectives.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return xent_loss + kl_loss

    return (vae_loss, Lambda(sampling, output_shape=(latent_rep_size,), name='lambda')([z_mean, z_log_var]))



In [ ]:
def _build_decoder(self, encoded, vocab_size, max_length):
    repeated_context = RepeatVector(max_length)(encoded)

    h = LSTM(500, return_sequences=True, name='dec_lstm_1')(repeated_context)
    h = LSTM(500, return_sequences=True, name='dec_lstm_2')(h)

    decoded = TimeDistributed(Dense(vocab_size, activation='softmax'), name='decoded_mean')(h)

    return decoded

In [ ]:
def _build_sentiment_predictor(self, encoded):
    h = Dense(100, activation='linear')(encoded)

    return Dense(1, activation='sigmoid', name='pred')(h)

In [86]:
MAX_LENGTH = 300
NUM_WORDS = 1000

In [87]:
from tensorflow.keras.datasets import imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

print("Training data")
print(X_train.shape)
print(y_train.shape)

print("Number of words:")
print(len(np.unique(np.hstack(X_train))))

17465344/17464789 [==============================] - 2s 0us/step
Training data
(25000,)
(25000,)
Number of words:
998


In [92]:
import keras
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, maxlen=MAX_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_LENGTH)

train_indices = np.random.choice(np.arange(X_train.shape[0]), 2000, replace=False)
test_indices = np.random.choice(np.arange(X_test.shape[0]), 1000, replace=False)

X_train = X_train[train_indices]
y_train = y_train[train_indices]

X_test = X_test[test_indices]
y_test = y_test[test_indices]

In [94]:
temp = np.zeros((X_train.shape[0], MAX_LENGTH, NUM_WORDS))
temp[np.expand_dims(np.arange(X_train.shape[0]), axis=0).reshape(X_train.shape[0], 1), np.repeat(np.array([np.arange(MAX_LENGTH)]), X_train.shape[0], axis=0), X_train] = 1

X_train_one_hot = temp

temp = np.zeros((X_test.shape[0], MAX_LENGTH, NUM_WORDS))
temp[np.expand_dims(np.arange(X_test.shape[0]), axis=0).reshape(X_test.shape[0], 1), np.repeat(np.array([np.arange(MAX_LENGTH)]), X_test.shape[0], axis=0), X_test] = 1

x_test_one_hot = temp

In [ ]:
def create_model_checkpoint(dir, model_name):
    filepath = dir + '/' + \
               model_name + "-{epoch:02d}-{val_decoded_mean_acc:.2f}-{val_pred_loss:.2f}.h5"
    directory = os.path.dirname(filepath)

    try:
        os.stat(directory)
    except:
        os.mkdir(directory)

    checkpointer = ModelCheckpoint(filepath=filepath,
                                   verbose=1,
                                   save_best_only=False)

    return checkpointer

In [ ]:
def train():
    model = VAE()
    model.create(vocab_size=NUM_WORDS, max_length=MAX_LENGTH)

    checkpointer = create_model_checkpoint('models', 'rnn_ae')

    model.autoencoder.fit(x=X_train, y={'decoded_mean': X_train_one_hot, 'pred': y_train},
                          batch_size=10, epochs=10, callbacks=[checkpointer],
                          validation_data=(X_test, {'decoded_mean': x_test_one_hot, 'pred':  y_test}))

### Training

In [87]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [88]:
model.fit(train_data, epochs=3, validation_data=test_data)


Epoch 1/3
108/108 [==============================] - 17s 158ms/step - loss: 0.6806 - accuracy: 0.6623 - val_loss: 0.5159 - val_accuracy: 0.7540
Epoch 2/3
108/108 [==============================] - 14s 129ms/step - loss: 0.4465 - accuracy: 0.8007 - val_loss: 0.4789 - val_accuracy: 0.7806
Epoch 3/3
108/108 [==============================] - 14s 131ms/step - loss: 0.3455 - accuracy: 0.8544 - val_loss: 0.4992 - val_accuracy: 0.7640


In [89]:
# model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=0.2)
# Save model
# model.save('s2s.h5')

In [109]:
# model.fit(train_data, epochs=3, validation_data=test_data)

In [90]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

     79/Unknown - 4s 45ms/step - loss: 0.4992 - accuracy: 0.7640
Eval loss: 0.499, Eval accuracy: 0.764


In [94]:
prediction = model.predict(test_data)

In [101]:
list(map(int, list(prediction[0])))

[4, 7, -4, -3, -5, -3, -4, -3, -2, -3]

### Saving Outputs

In [ ]:
model.save('path_to_my_model.h5')

# Recreate the exact same model purely from the file
new_model = keras.models.load_model('path_to_my_model.h5')


In [ ]:

# Export the model to a SavedModel
model.save('path_to_saved_model', save_format='tf')

# Recreate the exact same model
new_model = keras.models.load_model('path_to_saved_model')

# Check that the state is preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Note that the optimizer state is preserved as well:
# you can resume training where you left off.
